In [ ]:
# Importing Libraries.
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

In [ ]:
# Setting Device Agnostic Code.
device  = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
def accuracy_fn(y_true, y_pred):
    return (torch.eq(y_true, y_pred).sum().item() / len(y_pred)) * 100

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=(-45,45))
])

In [ ]:
# Downloading MNIST dataset.

train_data = datasets.MNIST(
    root="MNIST_dataset",
    train=True,
    download=False,
    transform=transform,
    target_transform=None
)

test_data = datasets.MNIST(
    root="MNIST_dataset",
    train=False,
    download=False,
    transform=transform
)

In [ ]:
train_data[0][0].shape

In [ ]:
classes = train_data.classes
classes

In [ ]:
# Visualizing a sample from training data.
image, label = train_data[0]
plt.imshow(image.squeeze(), cmap='gray')
plt.title(classes[label])
plt.show()

In [ ]:
# Plot more images
fig = plt.figure(figsize=(6,6))
rows, cols = 4, 4
for i in range(1, rows * cols + 1):
    img, label = train_data[i]
    fig.add_subplot(rows, cols, i)
    plt.imshow(img.squeeze(), cmap="gray")
    plt.title(classes[label])
    plt.axis(False)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
BATCH_SIZE = 16 # hyper parameter.

train_dataloader = DataLoader(train_data,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dataloader = DataLoader(test_data,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
len(train_dataloader), len(test_dataloader)

In [ ]:
train_feature_batch, train_label_batch = next(iter(train_dataloader))
train_feature_batch.shape, train_label_batch.shape

In [ ]:
# What it does look like after convolving an image?
idx = 1
image, label = train_feature_batch[idx], train_label_batch[idx]
conv = nn.Conv2d(
    in_channels=1,
    out_channels=16,
    kernel_size=3,
    stride=1,
    padding=1,
)
output_feature = conv(image)
fig = plt.figure(figsize=(6,6))
rows,cols = 4,4
for i in range(rows * cols):
    img = output_feature[i].detach().numpy()
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(img)
    plt.axis(False)

In [ ]:
# MNIST model.
class MNIST_MODEL_V1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=5,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv2 = nn.Conv2d(
            in_channels=5,
            out_channels=10,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.conv3 = nn.Conv2d(
            in_channels=10,
            out_channels=20,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.relu = nn.ReLU()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=20*28*28,
                      out_features=10)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.classifier(x)
        return x

model_1 = MNIST_MODEL_V1()
model_1.to(device)
model_1

In [ ]:
next(model_1.parameters()).device

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(), 
                            lr=0.01)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
from helper_functions import accuracy_fn

def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
def print_train_time(start, end, device):
    print(f"Time: {round(end-start, 3)} seconds on device: {device}")

In [ ]:
torch.manual_seed(55)

# Measure time
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 5
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}")
    train_step(data_loader=train_dataloader, 
        model=model_1, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=test_dataloader,
        model=model_1,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

train_time_end_on_gpu = timer()
total_train_time_model_1 = print_train_time(start=train_time_start_on_gpu,
                                            end=train_time_end_on_gpu,
                                            device=device)

In [ ]:
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, 
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        
        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [ ]:
torch.manual_seed(42)
result_model_1 = eval_model(
    model_1,
    test_dataloader,
    loss_fn,
    accuracy_fn,
)
result_model_1

In [ ]:
idx = 5
test_feature_batch, test_label_batch = next(iter(test_dataloader))
image, label = test_feature_batch[idx], test_label_batch[idx]
image, label = image.to(device), label.to(device)
model_1.eval()
with torch.inference_mode():
    y_pred = model_1(image.unsqueeze(dim=1))
    y_pred = y_pred.argmax(dim=1)
plt.imshow(image.to('cpu').squeeze())
plt.title(f'PRED:{y_pred.item()}||TRUTH:{label}')
plt.show()

In [ ]:
torch.save(obj=model_1.state_dict(), # only saving the state_dict() only saves the learned parameters
           f='MNIST_hand_digits_classification_model_1.pth')

In [ ]:
loaded_model = MNIST_MODEL_V1()
loaded_model.load_state_dict(torch.load('MNIST_hand_digits_classification_model_1.pth'))

In [ ]:
loaded_model.to(device)

In [ ]:
# Evaluate loaded model
torch.manual_seed(42)

loaded_model_2_results = eval_model(
    model=loaded_model,
    data_loader=test_dataloader,
    loss_fn=loss_fn, 
    accuracy_fn=accuracy_fn
)

loaded_model_2_results

In [ ]:
import cv2

In [ ]:
img = cv2.imread('MNIST_dataset/six.png')
img_scaled = cv2.resize(img, None, fx=2, fy=2)
cv2.imshow("14_IMAGE", img)
cv2.imshow("28_IMAGE", img)
cv2.imwrite('six_scaled.png', img_scaled)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# classification from image that is drawn in paint.
img_path = 'MNIST_dataset/my_hand_written.png'
img = cv2.imread(img_path, 0)
tensor_img = torch.Tensor(img).unsqueeze(dim=0).unsqueeze(dim=0)
tensor_img = tensor_img.div(255)
tensor_img = tensor_img.to(device)
loaded_model.eval()
with torch.inference_mode():
    y_pred = loaded_model(tensor_img)
    y_pred = y_pred.argmax(dim=1)
plt.imshow(img)
plt.title(f'Prediction: {classes[y_pred.item()]}')

In [ ]:
loaded_model.state_dict()